# Lego Database
Preguntas a responder:
- ¿Cómo evolucionaron los sets de lego en tamaño a través de los años?
- ¿Existe alguna asociación entre los colores y las temáticas?
- ¿Podría predecir a que temática pertenece un set basado en el
contenido de este?
- A través de los años, ¿Cuál o cuáles son los sets que tienen las piezas más
raras?
- ¿Cómo evolucionaron los colores en los sets de lego a través de los
años?

Dataset schema:

![dataset-schema](./imgs/downloads_schema.png)

In [4]:
# Dependencies
import pandas as pd
import seaborn as sns

# Global vars
DATSET_PATH = "./data/"

## Exploración y comprensión de los datos:
- Cargar el dataset proporcionado y realizar un análisis exploratorio de los datos.

In [ ]:
inventory_sets_df = pd.read_csv(DATSET_PATH + "inventory_sets.csv")
print("Amount of entries in inventory sets:", len(inventory_sets_df))
inventory_sets_df.head()

In [ ]:
inventory_parts_df = pd.read_csv(DATSET_PATH + "inventory_parts.csv")
print("Amount of entries in inventory parts:", len(inventory_parts_df))
inventory_parts_df.head()

In [ ]:
inventories_df = pd.read_csv(DATSET_PATH + "inventories.csv")
print("Amount of entries in inventories:", len(inventories_df))
inventories_df.head()

In [ ]:
colors_df = pd.read_csv(DATSET_PATH + "colors.csv")
print("Amount of entries in colors:", len(colors_df))
colors_df.head()

In [ ]:
part_categories_df = pd.read_csv(DATSET_PATH + "part_categories.csv")
print("Amount of entries in part categories:", len(part_categories_df))
part_categories_df.head()

In [ ]:
parts_df = pd.read_csv(DATSET_PATH + "parts.csv")
print("Amount of entries in parts:", len(parts_df))
parts_df.head()

In [ ]:
sets_df = pd.read_csv(DATSET_PATH + "sets.csv")
print("Amount of entries in sets:", len(sets_df))
sets_df.head()

In [ ]:
themes_df = pd.read_csv(DATSET_PATH + "themes.csv")
print("Amount of entries in themes:", len(themes_df))
themes_df.head()